In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('../src')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import unidecode
import dateparser

tqdm.pandas()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

import constants

| Kategori                                                               | İçerik Sayısı     | Hedef Değişkenler                                                                                            |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`,`mevzuat_tarihi`, `madde_sayisi`
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`
|
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | `kategori`, `donem` ve `sira_no`                                                                       |
| [Genelge](#kategori-genelge)                                           | 385               | `kategori`, `mevzuat_tarihi` ve varsa `mevzuat_no` yoksa `belge_sayi` 
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `madde_sayisi` ve varsa ` mevzuat_tarihi`
| [Tüzük](#kategori-tüzük)                                               | 82                |  `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum`, `madde_sayisi`       |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum` 
| [Özelge](#kategori-özelge)                                             | 750               | `kategori`, `mevzuat_tarihi`

In [5]:
df=pd.read_csv('../data/kanunum-nlp-doc-analysis-dataset.csv')

In [6]:
kanun=df[df.kategori=='Kanun']
khk=df[df.kategori=='Kanun Hükmünde Kararname']
resmi_gazete=df[df.kategori=='Resmi Gazete']
genelge=df[df.kategori=='Genelge']
c_kararname=df[df.kategori=='Cumhurbaşkanlığı Kararnamesi']
tüzük=df[df.kategori=='Tüzük']
yönetmelik=df[df.kategori=='Yönetmelik']
teblig=df[df.kategori=='Tebliğ']
ozelge=df[df.kategori=='Özelge']

In [7]:
class FeatureExtractor:
    def __init__(self):
        self.feature_name = None
        self.cols = []

    def _extractor_func(self, row_data):
        raise NotImplementedError()

    def extract(self, input_df):
        print(f"'{self.feature_name}' cikariliyor... -> '{self.feature_name}_pred'")
        feature_indexer = input_df.kategori.isin(self.cols)
        input_df[self.feature_name + '_pred'] = None
        input_df.loc[feature_indexer,
                     self.feature_name + '_pred'] = input_df[feature_indexer].progress_apply(
            lambda x: self._extractor_func(x),
            axis=1)

class RegaTarihiExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "rega_tarihi"
        self.cols = constants.COLS_REGA_TARIHI

    def _extractor_func(self, row_data):
        rega_tarihi = np.nan
        std_txt = unidecode.unidecode(row_data.data_text).lower()
        try:
            if row_data.kategori == 'Kanun':
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
            #######################################################
            elif row_data.kategori == 'Kanun Hükmünde Kararname':
                rega_tarihi =  std_txt.split("resmi gazete tarihi: ", 1)[1].split('resmi gazete sayisi:', 1)[0]
            #######################################################
            elif row_data.kategori == 'Resmi Gazete':
                std_txt = unidecode.unidecode(row_data.baslik).lower()
                rega_tarihi = std_txt.replace(' ','').split('tarihli',1)[0]
            #######################################################
            elif row_data.kategori == "Cumhurbaşkanlığı Kararnamesi":
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0].replace(" ","")
            #######################################################
            elif row_data.kategori == 'Tüzük':  
                rega_tarihi=std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0].replace(" ","")
            #######################################################
            elif row_data.kategori == 'Yönetmelik':
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
            #######################################################
            elif row_data.kategori == 'Tebliğ':
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
        except:
            pass
        rega_tarihi=str(pd.to_datetime(rega_tarihi,dayfirst=True).date())
        return rega_tarihi
    

class MevzuatTarihiExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "mevzuat_tarihi"
        self.cols = constants.COLS_MEVZUAT_TARIHI
    
    def _extractor_func(self, row_data):
        std_txt = unidecode.unidecode(row_data.data_text).lower()
        mevzuat_tarihi = np.nan
        try:
            if row_data.kategori == 'Kanun':
                mevzuat_tarihi = pd.to_datetime(std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[1]\
                                                    .split('kabul tarihi : ')[1].split('\n')[0],dayfirst=True)

            #######################################################
            elif row_data.kategori == 'Kanun Hükmünde Kararname':
                mevzuat_tarihi = pd.to_datetime(std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[1]\
                                                    .split('kararnamenin tarihi : ')[1].split('\n')[0],dayfirst=True)
            #######################################################
            elif row_data.kategori == 'Genelge':
                try:
                    std_txt=row_data.data_text
                    std_txt=std_txt.split('Tarih',1)[1].split('Sayı')[0].replace('\n','')
                    mevzuat_tarihi=pd.to_datetime(std_txt,dayfirst=True)
                except:
                    try:
                        std_txt=row_data.data_text
                        mevzuat_tarihi=pd.to_datetime(std_txt.split('Sayı',1)[0].split('Tarih')[1].replace('\n',''),dayfirst=True)
                    except:
                        try:
                            mevzuat_tarihi=pd.to_datetime(re.findall(r'\d{2}.\d{2}.\d{4}', std_txt.replace('-',' ')\
                                                                     .replace('/',''))[0],dayfirst=True)
                        except:
                            try:
                                mevzuat_tarihi=pd.to_datetime(re.findall(r'\d{2}/\d{2}/\d{4}', std_txt.replace('-',' '))[0],dayfirst=True)
                            except:
                                try:
                                    mevzuat_tarihi=pd.to_datetime(re.findall(r'\d{1}/\d{1}/\d{4}', std_txt)[0],dayfirst=True)
                                except:
                                    mevzuat_tarihi=np.nan
            #######################################################
            elif row_data.kategori == 'Cumhurbaşkanlığı Kararnamesi':
                mevzuat_tarihi = row_data.data_text.split('\n')[-1]
                mevzuat_tarihi = pd.to_datetime(dateparser.parse(mevzuat_tarihi,languages=['tr']),dayfirst=True)
            #######################################################
            elif row_data.kategori == 'Tüzük':
                mevzuat_tarihi = pd.to_datetime(std_txt.split("bkk no: ",1)[0].split("karar tarihi: ",1)[1].replace(",","")\
                                                .replace(" ",""),dayfirst=True)
            #######################################################
            elif row_data.kategori == 'Özelge':
                mevzuat_tarihi = std_txt.split("konu",1)[0].split("\n\n")
                try:
                    while '' in mevzuat_tarihi:
                        mevzuat_tarihi.remove('')
                    try:
                        mevzuat_tarihi=pd.to_datetime(mevzuat_tarihi[-1],dayfirst=True)
                    except:
                        try:
                            mevzuat_tarihi=pd.to_datetime(std_txt.split("tarih",1)[1].split('sayi')[0],dayfirst=True)
                        except:
                            try:
                                mevzuat_tarihi=pd.to_datetime(std_txt.split("sayi",1)[0].split('tarih')[1]\
                                                              .replace('\n','').replace(" ","").replace(":",""),dayfirst=True)
                            except:
                                try:
                                    mevzuat_tarihi=pd.to_datetime(std_txt.split('sayi')[0].split('tarih')[-1],dayfirst=True)
                                except:
                                    try:
                                        mevzuat_tarihi=pd.to_datetime(std_txt.split('*')[0].split(' ')[-1],dayfirst=True)
                                    except:
                                        try:
                                            mevzuat_tarihi=std_txt.split('konu')[0].replace('\n',"").split(' ')
                                            while '' in mevzuat_tarihi:
                                                mevzuat_tarihi.remove('')
                                            mevzuat_tarihi=pd.to_datetime(mevzuat_tarihi[-1],dayfirst=True)
                                        except:
                                            try:
                                                mevzuat_tarihi=pd.to_datetime(re.search(r'\d{2}.\d{2}.\d{4}', std_txt)\
                                                                              .group(),dayfirst=True)
                                            except:
                                                try:
                                                    mevzuat_tarihi=pd.to_datetime(re.search(r'\d{2}/\d{2}/\d{4}', std_txt)\
                                                                                  .group(),dayfirst=True)
                                                except:
                                                    mevzuat_tarihi=np.nan        
                
                except:
                    pass
        except:
            pass
        try:
            mevzuat_tarihi=str(mevzuat_tarihi.date())
        except:
            pass
            
        return mevzuat_tarihi
    

#class KurumExtractor(FeatureExtractor):
#    def __init__(self):
#        super().__init__()
#        self.feature_name = "kurum"
#        self.cols = constants.COLS_KURUM
#        self.kurumlar = constants.KURUMLAR
#    
#    def jaccard_similarity(x,y):
#        """İki list arasında jaccard benzerliğini hesaplar"""
#        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
#        union_cardinality = len(set.union(*[set(x), set(y)]))
#        return intersection_cardinality/float(union_cardinality)
#    
#    def _extractor_func(self, row_data):
#        std_text = row_data.data_text.split('Bu Tebliğ')[-1]
#        kurum = np.nan
#        
#
#        
#        if row_data.kategori == 'Tebliğ' :
#            items=[]
#            #std_text=std_text.split('Bu Tebliğ')[-1]
#            for item in constants.KURUMLAR:
#                if item in std_text:
#                    items.append(item)
#            if len(items)>1:
#                kurum=items[0]
#            elif len(items)==0:
#                kurum='Kurum'
#            elif len(items)==1:
#                kurum=items[0]
#            else:
#                kurum = np.nan
#
#        #######################################################
#        return kurum

In [15]:
#from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=mevzuat_tarihi_extractor,
                   include_na=True)

Veri Sayisi: 1876
Dogru Tahmin Sayisi: 1733
Yanlis Tahmin Sayisi: 143
Basari Orani: 0.9238


In [16]:
from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=rega_tarihi_extractor,
                   include_na=True)

Veri Sayisi: 1907
Dogru Tahmin Sayisi: 1907
Yanlis Tahmin Sayisi: 0
Basari Orani: 1.0


In [9]:
rega_tarihi_extractor = RegaTarihiExtractor()

In [10]:
mevzuat_tarihi_extractor = MevzuatTarihiExtractor()

In [11]:
rega_tarihi_extractor.extract(df)

'rega_tarihi' cikariliyor... -> 'rega_tarihi_pred'


100%|█████████████████████████████████████████████████████████████████████████████| 1907/1907 [00:10<00:00, 188.08it/s]


In [12]:
mevzuat_tarihi_extractor.extract(df)

'mevzuat_tarihi' cikariliyor... -> 'mevzuat_tarihi_pred'


100%|█████████████████████████████████████████████████████████████████████████████| 1876/1876 [00:12<00:00, 153.28it/s]


In [28]:
df.data_text[1178]

'Sayı :B.13.2.SGK.0.10.03.01/01 /1228 26 KASIM 2012\nKonu : Türkiye Almanya Sosyal Güvenlik\n\nSözleşmesinin hastalık sigortası hükümlerinin \n4/1 (b) ve 4/1 (c) yönünden uygulanması\n\nTC.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI\n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nGENELGE\n2012/38\n\nAlman Yasal Hastalık Sigortası Başkanlığı Hastalık Sigortası Yurtdışı İrtibat Mercii \nile vanlan mutabakat sonucunda 5510 sayılı Sosyal Sigortalar ve Genel Sağlık Sigortası \nKanunun 4 üncü maddesinin birinci fıkrasının (b), (c) bentleri ve geçici 4 üncü maddesi \nkapsamında sayılanlar ile emekli, gelir ya da aylık alanlar ve bakmakla yükümlü olduğu \nkişilerin, Türkiye-Almanya Sosyal Güvenlik Sözleşmesinin sağlık sigortası uygulamalarından \nyararlanabilmeleri kararlaştırılmıştır.\n\nBu kapsamdaki kişilerin Sözleşmenin sağlık sigortası uygulamalarından \nyararlanabilmeleri için yapılması gereken işlemler aşağıda açıklanmıştır.\n\n1. KANUNUN 4/1 (B) KAPSAMINDA SAYILANLAR\n\nKanunun 4/1 (b) kapsamındaki

In [24]:
df.mevzuat_tarihi[1123]

'2008-06-24'

In [27]:
pd.to_datetime(re.findall(r'\d{1}/\d{1}/\d{4}', df.data_text[1084])[0],dayfirst=True)

IndexError: list index out of range

In [22]:
re.findall(r'\d{2}.\d{2}.\d{4}', df.data_text[1084])

['11-10/8965', '11-10/4741', '11-10/5082', '11-10/4881']

In [17]:
df[(df.kategori=='Genelge')&df.mevzuat_tarihi_pred.isna()]

,id,kategori,baslik,rega_no,mukerrer_no,rega_tarihi,kurum,mevzuat_no,belge_sayi,mevzuat_tarihi,donem,sira_no,madde_sayisi,data_text,url,kanunum_url,rega_tarihi_pred,mevzuat_tarihi_pred
920,1829,Genelge,YENİDEN YAPILANDIRMA HAKKINDA GENELGE,NaN,0.0,NaN,Bankacılık Düzenleme ve Denetleme Kurumu,2018/1,NaN,2018-03-02,NaN,NaN,0,GENELGE\n\n\n2018/1 \n\nKredilerin Sınıflandır...,https://www.bddk.org.tr/Mevzuat/DokumanGetir/900,https://www.kanunum.com/content/21185252,None,NaN
1075,2118,Genelge,TÜRKİYE SERMAYE PİYASASI ARACI KURULUŞLARI BİR...,NaN,0.0,NaN,Türkiye Sermaye Piyasası Aracı Kuruluşları Bir...,16,NaN,2013-11-08,NaN,NaN,0,"İstanbul, 08 Kasım 2013 2013/967 \n\n\n\nGENEL...",http://www.tspakb.org.tr/tr/Default.aspx?tabid...,https://www.kanunum.com/content/918460,None,NaN
1081,2128,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1367 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1367,NaN,1962-06-09,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1367 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1367-sayili-i...,https://www.kanunum.com/content/1306116,None,NaN
1084,2133,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1467 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1467,NaN,1984-05-09,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1467 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1467-sayili-2...,https://www.kanunum.com/content/1306128,None,NaN
1085,2134,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1473 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1473,NaN,1985-01-08,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1473 Sayılı ...",www.tkgm.gov.tr/tr/icerik/1473-sayili-3083-say...,https://www.kanunum.com/content/1306130,None,NaN
1086,2135,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1476 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1476,NaN,1985-05-09,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1476 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1476-sayili-d...,https://www.kanunum.com/content/1306266,None,NaN
1087,2136,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1477 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1477,NaN,1985-11-07,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1477 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1477-sayili-3...,https://www.kanunum.com/content/1306268,None,NaN
1088,2139,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1481 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1481,NaN,1986-06-18,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1481 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1481-sayili-s...,https://www.kanunum.com/content/1306286,None,NaN
1089,2140,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1485 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1485,NaN,1988-04-18,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1485 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1485-sayili-7...,https://www.kanunum.com/content/1306288,None,NaN
1090,2141,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 1491 Sayılı ...",NaN,0.0,NaN,Tapu ve Kadastro Genel Müdürlüğü,1491,NaN,1988-11-17,NaN,NaN,0,"Tapu ve Kadastro Genel Müdürlüğü, 1491 Sayılı ...",http://www.tkgm.gov.tr/tr/icerik/1491-sayili-m...,https://www.kanunum.com/content/1306290,None,NaN


In [455]:
(df[df.kategori=='Özelge'].mevzuat_tarihi==df[df.kategori=='Özelge'].mevzuat_tarihi_pred).value_counts()

False    750
dtype: int64

In [456]:
(df[df.kategori=='Genelge'].mevzuat_tarihi==df[df.kategori=='Genelge'].mevzuat_tarihi_pred).value_counts()

False    385
dtype: int64